In [ ]:
import spacy

In [ ]:
nlp = spacy.load("en_core_web_sm")
print(nlp)

In [ ]:
nlp.pipe_names

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [ ]:
sentence = "European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices"
data = nlp(sentence)

from spacy import displacy
displacy.render(data, style="ent", jupyter=True)

In [ ]:
spacy.explain('NORP')

'Nationalities or religious or political groups'

In [ ]:
#Printing the named entity found by the model 
for w in data.ents:
    print(w.text,w.label_)

European NORP
$5.1 billion MONEY
Wednesday DATE


In [ ]:
#Inside-Outside-Beginning(IOB) Tagging
[(X, X.ent_iob_, X.ent_type_) for X in data if X.ent_type_]

[(European, 'B', 'NORP'),
 ($, 'B', 'MONEY'),
 (5.1, 'I', 'MONEY'),
 (billion, 'I', 'MONEY'),
 (Wednesday, 'B', 'DATE')]

In [ ]:
#the text was taken in 
#https://www.livemint.com/money/personal-finance/union-bank-of-india-revises-interest-rate-on-fixed-deposits-of-less-than-rs-2-cr-11663318976878.html
#using annotate tool json file was created

In [ ]:
import json
f = open('C:\\Users\\Nandhini.karuppusamy\\Desktop\\data.json')
data = json.load(f)

In [ ]:
print(data)

{'classes': ['ORG', 'MONEY', 'DATE', 'DAYS', 'YEARS', 'PERCENT'], 'annotations': [["Union Bank of India, a public sector lender, has modified the interest rates on fixed deposits below â‚¹2 crore. According to the bank's official website, the new interest rates are effective as of 14 September 2022.\r", {'entities': [[0, 19, 'ORG'], [101, 110, 'MONEY'], [196, 213, 'DATE']]}], ['Following the revision, the bank is now providing an interest rate on fixed deposits with maturities ranging from 7 days to 10 years of 3.00% to 5.80 per cent. \r', {'entities': [[114, 120, 'DAYS'], [124, 132, 'YEARS'], [136, 141, 'PERCENT'], [145, 158, 'PERCENT']]}], ['Currently, Union Bank of India is providing a maximum interest rate of 6.20% on fixed deposits maturing in 5 Years, 1 Day.\r', {'entities': [[11, 30, 'ORG'], [71, 76, 'PERCENT'], [107, 114, 'YEARS'], [116, 121, 'DAYS']]}], ['On fixed deposits maturing in 7 days to 45 days the bank will now offer an interest rate of 3.00% and on term deposits matu

In [ ]:
#Conversion of Data to .spacy format

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object
for text, annot in tqdm(data['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

100%|██████████| 10/10 [00:00<?, ?it/s]


In [ ]:
#Creating the Config file for training the model

In [ ]:
#Run the Training configuration

In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy 


[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     50.92    0.00    0.00    0.00    0.00
 43     200         66.80   1413.98  100.00  100.00  100.00    1.00
100     400          0.00      0.00  100.00  100.00  100.00    1.00
167     600          0.00      0.00  100.00  100.00  100.00    1.00
253     800         18.65     10.66  100.00  100.00  100.00    1.00
353    1000          8.64      4.70  100.00  100.00  100.00    1.00
453    1200          0.00      0.00  100.00  100.00  100.00    1.00
642    1400          6.57      3.52  100.00  100.00  100.00    1.00
842    1600          

2022-09-17 23:48:23.585888: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-09-17 23:48:23.585921: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-17 23:48:26.376776: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-09-17 23:48:26.376802: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-17 23:48:26.382300: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: KANINI-LTP-956
2022-09-17 23:48:26.382443: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: KANINI-LTP-956
[2022-09-17 23:48:27,116] [INFO] Set up nlp object from config
[2022-09-17 23:48:27,124] [INFO] Pipeline: ['

In [ ]:
nlp = spacy.load("output/model-last/")

In [ ]:
sentence = "Union Bank of India Q1 net profit up 32% "
doc = nlp(sentence)

from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)